In [21]:
import pandas as pd
import numpy as np
from lib import *

In [22]:
# Read data of BFRSS about Crude Prevalence 
typeOfTarget = 'CP'
data =  pd.read_csv("Datasets/Diabetes_"+typeOfTarget+"_allYear_overall.csv",sep = ';',encoding = "utf-8-sig")
# Clean data an format the name of the columns
mylist = ['All States and DC (median) **', 'All States, DC and Territories (median) **', 'Guam', 
        'Puerto Rico', 'Virgin Islands']
data.rename(columns = {'Locationdesc': 'State',
                     'Confidence_limit_Low':'Lower_CI',
                     'Confidence_limit_High':'Upper_CI',
                     },inplace = True)
data = data[~data.State.isin(mylist)].reset_index(drop = True)
# Change the type of years in string for multiIndex
data.Year = data.Year.astype(str)
# Reorder columns
data = data[["State","Year","Data_value",'Lower_CI','Upper_CI','Sample_Size']]
#data.to_csv('Crude_prevalence_diabetes_BFRSS.csv')

In [23]:
# Read data of BFRSS about Age-Adjusted Prevalence 
typeOfTarget = 'APP'
data =  pd.read_csv("Datasets/Diabetes_"+typeOfTarget+"_allYear_overall.csv",sep = ';',encoding = "utf-8-sig")
# Clean data an format the name of the columns
mylist = ['All States and DC (median) **', 'All States, DC and Territories (median) **', 'Guam', 
        'Puerto Rico', 'Virgin Islands']
# Add by hand Rhode Island for the years 2011 and 2012
data.rename(columns = {'Locationdesc': 'State',
                     'Low_Confidence_Limit':'Lower_CI',
                      'High_Confidence_Limit':'Upper_CI'
                      },inplace = True)
data = data[~data.State.isin(mylist)].reset_index(drop = True)
# Change the type of years in string for multiIndex
data.Year = data.Year.astype(str)
# Reorder columns
data = data[["State","Year","Data_value",'Lower_CI','Upper_CI','Sample_Size']]
#data.to_csv('AgeAdjusted_prevalence_diabetes_BFRSS.csv')

In [ ]:
from pytrends.request import TrendReq
# Unofficial Google Trends API for download the keywords
pytrend  =  TrendReq()
kws = [['Corticosteroids ', 'Obstructive sleep apnea ','Blood glucose ','Glucose levels','Hyperglycemia'],
    ['2diabetes','Type2','diabetes','Hypertension','Diabetesin','Type2'],
    ['Insulin','Glucose','HeartDisease','Cholesterol','Diabetes']]

for kw in kws:
    for t in range(2004,2017):
        # Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
        pytrend.build_payload(kw_list  =  kw, 
                              cat  =  45, 
                              timeframe  =  str(t)+'-01-01'+' '+str(t)+'-12-31', 
                              geo = 'US', 
                              gprop = '')

        # Interest by Region
        interest_by_region_df  =  pytrend.interest_by_region()
        name = "".join([word.replace(" ","") for word in kw])
        #interest_by_region_df.to_csv(name+'_'+str(t)+'.csv')

In [25]:
# Read data of google trends and takes keyword
per = ['gt/InsulinGlucoseHeartDiseaseCholesterolDiabetes_',
      'gt/2diabetesType2diabetesHypertensionDiabetesinType2_',
    'gt/CorticosteroidsObstructivesleepapneaBloodglucoseGlucoselevelsHyperglycemia_']

#define the start year for Crude Prevalence
start_year = 2004
data =  pd.read_csv("Datasets/Crude_prevalence_diabetes_BFRSS.csv",sep = ',',encoding = "utf-8-sig",index_col = 0)
# Take states
states = sorted( list( set( data.State)))
# Take year
years = list(range(start_year,2017))
#Union of BRFSS data Crude Prevalence & google trends in a single dataset with primary key the tuple (State,year) 
df_gt,keywords = gt_dataset(per,states,start_year)
df = data[["State","Year"]]
for kw in keywords:
    col = np.array(())
    for y in reversed(years):
        col = np.concatenate( ( col,df_gt[(str(kw) +':'+ str(y))].values )) 
    df[kw] = col
df["y"] =  data.loc[:,"Data_value"]
#df.to_csv('Crude_prevalence_diabetes_BFRSS_union_GT.csv')

#define the start year for Age-Adjusted Prevalence
start_year = 2011
data =  pd.read_csv("Datasets/AgeAdjusted_prevalence_diabetes_BFRSS.csv",sep = ',',encoding = "utf-8-sig",index_col = 0)
# Take states
states = sorted( list( set( data.State)))
# Take year
years = list(range(start_year,2017))
# Union of BRFSS data Crude Prevalence & google trends in a single dataset with primary key the tuple (State,year)
df_gt,keywords = gt_dataset(per,states,start_year)
df = data[["State","Year"]]
for kw in keywords:
    col = np.array(())
    for y in reversed(years):
        col = np.concatenate( ( col,df_gt[(str(kw) +':'+ str(y))].values )) 
    df[kw] = col
df["y"] =  data.loc[:,"Data_value"]
#df.to_csv('AgeAdjusted_prevalence_diabetes_BFRSS_union_GT.csv')

C:\Users\Emmanuele\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Emmanuele\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
# Create a dataset that contain info about the target value (Crude Prevalence) like CI and sample size
start_year = 2004
data =  pd.read_csv("Datasets/Crude_prevalence_diabetes_BFRSS.csv",sep = ',',encoding = "utf-8-sig",index_col = 0)
df_info = data[["State","Year",'Lower_CI','Upper_CI', 'Sample_Size']]
#df_info.to_csv('Crude_prevalence_INFO_diabetes_BFRSS_union_GT.csv')

# Create a dataset that contain info about the target value (Age-Adjusted Prevalence) like CI and sample size
start_year = 2011
data =  pd.read_csv("Datasets/AgeAdjusted_prevalence_diabetes_BFRSS.csv",sep = ',',encoding = "utf-8-sig",index_col = 0)
df_info = data[["State","Year",'Lower_CI','Upper_CI', 'Sample_Size']]
#df_info.to_csv('AgeAdjusted_prevalence_INFO_diabetes_BFRSS_union_GT.csv')

In [27]:
# Create a dataset about Income from 2004 to 2016
df_income =  pd.read_csv("Datasets/Income_CP_allYear_lessthan15000.csv",sep = ';',encoding = "utf-8-sig")
# Clean data an format the name of the columns
df_income.rename(columns = {'Locationdesc': 'State',
                      'Data_value' : 'IncomePercent'},inplace = True)
mylist = ['Guam','Puerto Rico', 'Virgin Islands']
# Add by hand Hawaii in 2004
df_income = df_income[['Year','State','IncomePercent']]
df_income = df_income[~df_income.State.isin(mylist)].reset_index(drop = True)
#df_income.to_csv("Income_2004_2016_AllStates.csv")

In [28]:
# Create a dataset about Poverty from 2004 to 2016
table  =  pd.read_csv('Datasets/Poverty_allYear.csv', sep = ';',encoding = 'UTF-8-sig ')
# Clean data an format the name of the columns
mylist = ['STATE','2004 (14)','2005','2006','2007','2008','2009','2010 (17)','2011','2012','2013 (18)','2013 (19)','2014','2015']
table = table[~table.STATE.isin(mylist)].reset_index()
table = table[['STATE','Percent']]
l = []
for year in reversed(range(2004,2017)):
    l.extend(np.repeat(str(year),51).tolist())
table.rename(columns = {'STATE': 'State' , 'Percent' : 'PovertyPercent' },inplace = True)
table['Year'] = l
table = table[['State','Year','PovertyPercent']] # Change the order
table.State = table.State.replace("D.C.","District of Columbia",regex = True)
table.PovertyPercent = table.PovertyPercent.replace(",",".",regex = True).astype(float)
#table.to_csv('Poverty_2004_2016_AllStates.csv')

In [29]:
df = pd.read_csv('Datasets/insurance_2004_2012.csv',header = 2,encoding = 'utf-8-sig',sep = ';')
# Clean data an format the name of the columns
states = [i.replace(":","") for i in df['Unnamed: 0'].unique()[15:].tolist()]
insurance_list = []
year_list = []
for i in reversed(range(2004,2013)):
    insurance_list.extend(df[df['Unnamed: 0'] == str(i)]['Percent.1'].values.tolist()[1:]) 
    year_list.extend(np.repeat(i,51))

insurance_dict  =  {'Year': year_list,
                  'InsurancePercent': insurance_list,
                  'State':states*9}
insurance_df  =  pd.DataFrame.from_dict(insurance_dict)
insurance_df = insurance_df[['Year','State','InsurancePercent']]


df1 = pd.read_csv('Datasets/insurance_2013_2016.csv')
df1[df1.columns[1:]] = 100-df1[df1.columns[1:]]
insurance_df2 = pd.DataFrame()
for i in reversed(range(2013,2017)):
    d = { "Year" :np.repeat(i,51) , "State" : states , "InsurancePercent" : df1[str(i)].values}   
    df_col = pd.DataFrame(d)
    insurance_df2  =  pd.concat([insurance_df2,df_col],axis = 0)
insurance_df2 = insurance_df2[['Year','State','InsurancePercent']]


df_insurance_complete = pd.concat([insurance_df2,insurance_df],axis = 0)
df_insurance_complete.InsurancePercent = df_insurance_complete.InsurancePercent.replace(",",".",regex = True).astype(float)
#df_insurance_complete.to_csv('Insurance_2004_2016_AllStates.csv')